# SLC45A2
[Albinism type IV (OCA4)](https://omim.org/entry/606574) is caused by homozygous or compound heterozygous mutation SLC45A2.

In [1]:
import gpsea
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2023-10-09')
print(f'Loaded HPO v{hpo.version}')
print(f"Using genophenocorr version {gpsea.__version__}")

Loaded HPO v2023-10-09
Using genophenocorr version 0.6.1.dev0


# SLC45A2
We user the [Matched Annotation from NCBI and EMBL-EBI (MANE)](https://www.ncbi.nlm.nih.gov/refseq/MANE/) transcript and the corresponding protein identifier for SLC45A2.

In [2]:
gene_symbol = 'SLC45A2'
mane_tx_id = 'NM_016180.5'
mane_protein_id = "NP_057264.4" #membrane-associated transporter protein isoform

In [3]:
from ppktstore.registry import configure_phenopacket_registry
from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets

from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store("0.1.20") as ps:
    phenopackets = tuple(ps.iter_cohort_phenopackets(gene_symbol))
len(phenopackets)


cohort_creator = configure_caching_cohort_creator(hpo)
cohort, validation = load_phenopackets(
    phenopackets=phenopackets, 
    cohort_creator=cohort_creator,
)

validation.summarize()

Individuals Processed: 100%|██████████| 30/30 [00:00<00:00, 36.33individuals/s]
Validated under permissive policy
No errors or warnings were found


In [ ]:
from gpsea.view import CohortViewer
cv = CohortViewer(hpo=hpo)
cv.process(cohort=cohort, transcript_id=mane_tx_id)

HPO Term,ID,Seen in n individuals
Hypopigmentation of hair,HP:0005599,30
White eyelashes,HP:0002227,29
White eyebrow,HP:0002226,29
Nystagmus,HP:0000639,28
Iris hypopigmentation,HP:0007730,28
Reduced visual acuity,HP:0007663,26
Hypoplasia of the fovea,HP:0007750,26
Iris transillumination defect,HP:0012805,24
Melanocytic nevus,HP:0000995,12
Nevus,HP:0003764,10


In [7]:
# Note we do not show the strcture because no domains/motifs are shown in UniProt (Q9UMX9).

In [8]:
from gpsea.view import CohortVariantViewer

viewer = CohortVariantViewer(tx_id=mane_tx_id)
viewer.process(cohort)

Variant key,Variant (cDNA),Variant (protein),Overlapping Exons,Effects,Count
5_33954406_33954407_TG_T,c.986del,p.Thr329LysfsTer69,4,frameshift,12
5_33944770_33944770_C_T,c.1471G>A,p.Gly491Arg,7,missense,4
5_33947363_33947365_CTT_C,c.1166_1167del,p.Lys389SerfsTer55,6,frameshift,3
5_33947257_33947258_AG_A,c.1273del,p.Leu425TrpfsTer9,6,frameshift,2
5_33944709_33944709_G_T,c.1532C>A,p.Ala511Glu,7,missense,2
5_33984325_33984326_CG_C,c.258del,p.Val87TrpfsTer26,1,frameshift,2
5_33944723_33944723_G_A,c.1518C>T,p.Val506=,7,synonymous,2
5_33954443_33954443_T_C,c.950A>G,p.Tyr317Cys,4,missense,2
5_33954363_33954365_GGC_G,c.1028_1029del,p.Gly343AlafsTer10,4,frameshift,2
5_33951674_33951674_C_A,c.1036G>T,p.Val346Leu,5,missense,2


In [9]:
from gpsea.analysis.predicate.phenotype import prepare_predicates_for_terms_of_interest

pheno_predicates = prepare_predicates_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
)
len(pheno_predicates)
from gpsea.analysis.mtc_filter import HpoMtcFilter
mtc_filter = HpoMtcFilter.default_filter( hpo=hpo)
mtc_correction = 'fdr_bh'
mtc_alpha = 0.05
from gpsea.analysis.pcats.stats import FisherExactTest

count_statistic = FisherExactTest()
from gpsea.analysis.pcats import HpoTermAnalysis

analysis = HpoTermAnalysis(
    count_statistic=count_statistic,
    mtc_filter=mtc_filter,
    mtc_correction=mtc_correction,
     mtc_alpha=mtc_alpha,
)

In [10]:
from gpsea.model import VariantEffect
from gpsea.analysis.predicate.genotype import VariantPredicates, biallelic_predicate

missense = VariantPredicates.variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=mane_tx_id)


gt_predicate = biallelic_predicate(a_predicate=missense, 
                                   b_predicate=~missense, 
                                   a_label="missense",
                                   b_label="other",
                                    partitions=((0,1),(2,))
                                   )
print(gt_predicate.display_question())
result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=gt_predicate,
    pheno_predicates=pheno_predicates,
)
from gpsea.view import MtcStatsViewer

viewer = MtcStatsViewer()
viewer.process(result)

Allele group: missense/missense OR missense/other, other/other


Code,Reason,Count
HMF01,Skipping term with maximum frequency that was less than threshold 0.4,2
HMF03,Skipping term because of a child term with the same individual counts,13
HMF08,Skipping general term,19
HMF09,Skipping term with maximum annotation frequency that was less than threshold 0.4,5


In [11]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=result)

Allele group                                    missense/missense OR missense/other  \
                                                                              Count   
Iris transillumination defect [HP:0012805]                                    13/17   
Esotropia [HP:0000565]                                                         4/16   
Heterotropia [HP:0032012]                                                      6/18   
Strabismus [HP:0000486]                                                        6/18   
Nystagmus [HP:0000639]                                                        16/18   
Iris hypopigmentation [HP:0007730]                                            16/18   
Melanocytic nevus [HP:0000995]                                                 9/14   
Nevus [HP:0003764]                                                            13/18   
Localized skin lesion [HP:0011355]                                            13/13   
Abnormal iris pigmentation [HP:0008034]                                       17/17   
White eyelashes [HP:0002227]                                                  17/17   
Abnormality of hair pigmentation [HP:0009887]                                 18/18   
Abnormal ocular adnexa morphology [HP:0030669]                                17/17   
White eyebrow [HP:0002226]                                                    17/18   
Abnormal eyebrow morphology [HP:0000534]                                      17/17   
Reduced visual acuity [HP:0007663]                                            16/16   
Abnormal involuntary eye movements [HP:0012547]                               16/16   
Hypopigmentation of hair [HP:0005599]                                         18/18   
Hypoplasia of the fovea [HP:0007750]                                          15/15   
Abnormal fundus morphology [HP:0001098]                                       15/15   
Abnormal retinal morphology [HP:0000479]                                      15/15   
Abnormal macular morphology [HP:0001103]                                      15/15   
Abnormal conjugate eye movement [HP:0000549]                                    6/6   
Abnormality of skin pigmentation [HP:0001000]                                   9/9   

Allele group                                            other/other          \
                                                Percent       Count Percent   
Iris transillumination defect [HP:0012805]          76%       11/11    100%   
Esotropia [HP:0000565]                              25%        5/10     50%   
Heterotropia [HP:0032012]                           33%        7/12     58%   
Strabismus [HP:0000486]                             33%        7/12     58%   
Nystagmus [HP:0000639]                              89%       12/12    100%   
Iris hypopigmentation [HP:0007730]                  89%       12/12    100%   
Melanocytic nevus [HP:0000995]                      64%         3/6     50%   
Nevus [HP:0003764]                                  72%        9/12     75%   
Localized skin lesion [HP:0011355]                 100%         9/9    100%   
Abnormal iris pigmentation [HP:0008034]            100%       12/12    100%   
White eyelashes [HP:0002227]                       100%       12/12    100%   
Abnormality of hair pigmentation [HP:0009887]      100%       12/12    100%   
Abnormal ocular adnexa morphology [HP:0030669]     100%       12/12    100%   
White eyebrow [HP:0002226]                          94%       12/12    100%   
Abnormal eyebrow morphology [HP:0000534]           100%       12/12    100%   
Reduced visual acuity [HP:0007663]                 100%       10/10    100%   
Abnormal involuntary eye movements [HP:0012547]    100%       12/12    100%   
Hypopigmentation of hair [HP:0005599]              100%       12/12    100%   
Hypoplasia of the fovea [HP:0007750]               100%       11/11    100%   
Abnormal fundus morphology [HP:0001098]            100%       11/11    100%   